<a href="https://colab.research.google.com/github/mertyyanik/Natural-Language-Processing-Real-or-Fake/blob/master/News.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Doğal Dil İşleme ve Derin Öğrenme - Gerçek ya da Sahte?
Doğal Dil işleme (Natural Language Processing), basit hali ile yapay zeka ve dil biliminin birleşmesi ile ortaya çıkmış bir kavramdır. Bu kavram herkesin günlük konuşma dilinin bilgisayara öğretilmesi ve bazı çalışmaların bilgisayara yaptırılmasına sebep olmuştur. Günümüze yakın tarihlere baktığımız zaman bilgisayarların güçlenmesini takip eden makine öğrenmesi modelleri daha da gelişmiş ve bir çok alanda kullanılmaya başlanmıştır. Bu alanlardan biriside doğal dil işlemedir. Doğal dil işlemeyi makine öğrenmesi modelleri ile birleştirdiğimiz zaman ortaya bir cümlenin kime ait olduğunu ya da bir cümleyi söyleyen kişinin nasıl bir ruh haline sahip olduğunu anlayabilir noktaya gelmiş bulunmaktayız.

Bizimde amacımız buna benzer bir çalışma yapmaktır. Elimizde haber sitelerinin yazmış olduğu haber başlıkları ve bu başlıkların gerçek ya da sahte olduğunu gösteren bir veri kümesi bulunmaktadır. Derin öğrenme yöntemlerini kullanarak bu veri kümesini bilgisayara öğretecek ve gelecekte yazılmış bir haber başlığının gerçek mi yoksa sahte mi olduğunu anlamaya çalışacağız.

Bunun için Google'ın geliştirmiş olduğu Colab ortamını kullanıyoruz. Bu ortamın avantajı derin öğrenme yöntemlerinde uygulamaları eğitirken **TPU**'ları kullanıyor olmasıdır. TPU'lar, GPU veya CPU'ya göre 15 kat daha hızlıdır. Bu da bizi büyük bir oranda zaman maliyetinden kurtarmaktadır.

Öncelikle elimizde ki veri kümelerini Colab ortamına aktarmamız gerekiyor. Bunu yapmadan önce ortamın Google Drive ile olan bağlantısını onaylamamız lazım. Aşağıda ki kod bloğunu çalıştırdığımız zaman ekranda bir link belirmektedir bu linke tıklayarak karşımıza çıkan kodu output kısmına yapıştırarak Google Drive ile olan bağlantıyı sağlamış oluyoruz.

In [0]:
!pip install -U -q PyDrive
 
from pydrive.auth import GoogleAuth
 
from pydrive.drive import GoogleDrive
 
from google.colab import auth
 
from oauth2client.client import GoogleCredentials
 
auth.authenticate_user()
 
gauth = GoogleAuth()
 
gauth.credentials = GoogleCredentials.get_application_default()

drive = GoogleDrive(gauth)

Bundan sonraki aşama veri kümelerini Google Drive'dan indirmektir. Burada ki önemli nokta dosya numarasının (File ID) elde edilmesidir. Bu numaraya  Google Drive üzerinden veri kümesinin paylaş butonuna basarak ulaşabilirsiniz.

In [0]:
#Test.csv Dosyası
from google.colab import files

file_id = '1Zx7P0t9lQLP4aj4YOUaZDxNRrSkZOTk4'

downloaded = drive.CreateFile({'id': file_id})

downloaded.GetContentFile('test.csv')

#Test.txt Dosyası
from google.colab import files
 
file_id = '1S2-N2HSIgtZpSzPmAEZWmVxV2YlOzCzs'
 
downloaded = drive.CreateFile({'id': file_id})
 
downloaded.GetContentFile('test.txt')

#clean_real-Train.txt Dosyası
from google.colab import files
 
file_id = '1pnsf7izWzNYlA41xcWrCXQ5SSFe5JCUF'
 
downloaded = drive.CreateFile({'id': file_id})
 
downloaded.GetContentFile('clean_real-Train.txt')

#clean_fake-Train.txt Dosyası
from google.colab import files
 
file_id = '1Mc1m6S6WCJuRkMBWM9_LZGHSzJIw2-e0'
 
downloaded = drive.CreateFile({'id': file_id})
 
downloaded.GetContentFile('clean_fake-Train.txt')


Bu noktadan sonra artık kendi Python kodlamanızı istediğiniz gibi yazabiliriz.

Öncelikle gerekli kütüphaneleri yükleyelim.

In [0]:
import pandas as pd
import numpy as np
import random
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn import metrics
from sklearn.metrics import accuracy_score
import time

Text dosyasında bulunan veri dosyalarımızı ayrı ayrı listelere aktarıyoruz. Böylece Python'ın nimetlerinden faydalanabileceğiz.

In [0]:
# Reading Data
input1Data = open('clean_real-Train.txt', 'r+')
input2Data = open('clean_fake-Train.txt', 'r+')
testData = open('test.txt')

testDataForComparison = pd.read_csv('test.csv').iloc[:, 1:]

input1List = []
input2List = []
testList = []

read = input1Data.readline()
read2 = input2Data.readline()
read3 = testData.readline()
rowCount = 0
while read != "":
    input1List.append(read)
    read = input1Data.readline()
    
while read2 != "":
    input2List.append(read2)
    read2 = input2Data.readline()

while read3 != "":
    testList.append(read3)
    read3 = testData.readline()
 

Daha sonra iki farklı veri kümesinde bulunan 'real' ve 'fake' veri kümelerini birleştiriyoruz. Real olanlara 1, fake olanlara 0 yazıyoruz.

In [0]:
# Real and Fake data combined.    
liste = ['real' for i in range(len(input1List))]
realInput = np.column_stack((input1List, liste))


liste = ['fake' for i in range(len(input2List))]
fakeInput = np.column_stack((input2List, liste))

mainInput = realInput.tolist() + fakeInput.tolist()
random.shuffle(mainInput)

sentenceInput = []
targetInput = []
targetInputFloat = []
for i in range(len(mainInput)):
    sentenceInput.append(mainInput[i][0])
    
for i in range(len(mainInput)):
    targetInput.append(mainInput[i][1])
    
from sklearn.preprocessing import LabelEncoder
lb = LabelEncoder()
targetInputFloat = lb.fit_transform(targetInput)

## Veri Ön İşleme
Bundan sonraki aşama veri ön işlemenin başladığı kısım. Veri kümesini bilgisayarın anlayabileceği sayılara çevirmeden önce bakıldığı zaman görülen bazı yanlışlar yok mu? Tabiki var. Bu yanlışların ne olduğundan bahsetmemiz ve en nihayetinde bunlardan kurtulmamız lazım. Yani veri kümemizi bazı veri ön işleme aşamalarından geçirmemiz gerekiyor. Bu aşamaları birkaç adımda sıralayabiliriz.
1. Veri kümesi içerisinde noktalama işaretleri kelimelerin bir parçası olmadığı için veri kümesinden silinmelidir. İleride bahsedilecek ama biz temelinde kelimeleri 0 ve daha farklı sayılardan oluşan bir matris haline getirmeye çalışıyoruz. Burada eğer noktalama işaretlerini yok etmezsek bilgisayar bunları da bir sayı olarak düşünür ve yanıltıcı bir durum ortaya çıkar. Bu istediğimiz bir durum değil.
2. İlk harfi büyük olan  Mehmet ile ilk harfi küçük olan mehmet farklı anlamamı gelmeli? Tabiki hayır. Bu nedenle elimizde ki veri kümesinde büyük harfleri küçük harflere çevirerek bu ayrımı ortadan kaldırmış oluyoruz.
3. Tıpkı büyük harf küçük harf durumunda olduğu gibi Ahmet’in ya da Ahmet’e gibi kelimelerin birbirinden farklılığı olmaması gerekir. Bu nedenle bir kelimenin başına veya sonuna gelen bütün eklerden kurtulmuş olması gerekir. Yani kelimelerin sadece köklerden oluşmasını isteriz.
4. Birleştirici Kelimeler. Bu durumunda diğerlerinden hiçbir farkı yok. ‘ve’, ‘veya’, ‘ile’ gibi bağlaçların silinmesi gerekir. Burada bütün adımlarda yaptığımız temel amaç aynıdır. Bu adımların sonunda geriye sadece kelimelerin kökleri kalmalıdır. Bunun dışında hiçbir şey istemiyoruz. Şimdi gelin bu işlerin Python kodlamasında nasıl yapıldığına bir bakalım.


In [0]:
# Stop Words
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

# Stemmer
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()

# Regular Expressions
import re

sentenceList = []
for i in range(len(sentenceInput)):
    # Regular Expressions
    comment = re.sub('[^a-zA-Z]', ' ', sentenceInput[i]) #-> 1.Adım
    # Upper Case Problem
    comment = comment.lower() #-> 2. Adım
    # Splitting
    comment = comment.split() #-> 3.Adım
    # Stemming and Stop Words
    comment = [ps.stem(kelime) for kelime in comment if not kelime in set(stopwords.words('english'))] # -> 4. Adım
    
    comment = ' '.join(comment)
    sentenceList.append(comment)
    
temporaryResult = pd.DataFrame(data=sentenceList)

Burada ilk 3 adım gerekli kütüphanelerin çağrıldığı kısımdır. Bizi asıl ilgilendiren ‘for’ döngüsünün başladığı kısım. Bu döngünün amacı her cümleye tek tek erişmektir. Yani bu döngü her çalıştığında biz bir satırda ki cümleye ulaşıyoruz. Dolayısıyla o cümleye ulaştığımıza göre her şeyi tek bir döngü içerisinde yapabiliriz.

**1.Adım(Regular Expressions) ->** İlk aşamada düzenli ifadeler(Regular Expressions) denilen bir kavram ortaya çıkıyor. Bu kavram bilgisayar bilimlerinde çok ayrıntılı bir şekilde anlatılan ve esasında bir programlama dili oluştururken o dile kurallar verdiğimiz kısımda kendini gösteriyor. Burada tek bilmemiz gereken şey 2 büyük parantez içerisinde ki ifadede a-z bütün küçük harflerin, A-Z bütün büyük ifadelerin, tırnak işaretinin bütün küçük harflerin ve bütün büyük harflerin dışında kalan ifadeleri temsil ediyor olmasıdır. İkinci kısımda ki boşluk karakteri, bu ifadelerden herhangi biriyle karşılaşılır ise o ifadeyi boşluk karakteri ile yer değiştirecektir. Bunu yaptığımız zaman bütün noktalama işaretlerinden kurtulmuş oluyoruz.

**2.Adım(Upper Case Problem) ->** Hatırlarsanız ikinci adımımız büyük harflerden kurtulmaktı. Burada da ikinci adımda lower() fonksiyonunu kullanarak bütün büyük harfleri küçük harfler ile yer değiştirmiş olduk.

**3.Adım(Splitting) ->** Daha sonrasında yapmamız gereken şey kelime eklerinden ve bağlaç gibi cümleden bağımsız olan kelimelerden kurtulmak. Bunu yapmadan önce elimizde her bir kelimenin olduğu bir liste olması gerekiyor. Split() fonksiyonu cümledeki her bir kelimeyi bir listeye aktarıyor. Şimdi artık bu kelimelerin her birine tek tek ulaşarak o kelimenin üzerinde işlem yapabiliriz.

**4.Adım(Stemming and Stop Words) ->** Bir alt satırda ki kod parçasında bir döngü ile her bir kelimeye tek tek ulaşıyor ve eğer bu kelime bir stopwords değilse o zaman stem() fonksiyonu ile başında veya sonunda bulunan eklerden kurtulmasını sağlıyoruz. Eğer bir stopwords ise o zaman kelimeyi tamamen yok ediyoruz.
Bütün bu adımların sonunda artık elimizde o 4 adımda istediğimiz sadece köklerden oluşan bir veri kümesi olması gerekiyor. Bakalım gerçekten öyle olmuş mu?

In [7]:
print(temporaryResult)

                                                      0
0                    trump famili say trump brand damag
1     donald trump unannounc visit korean dmz abando...
2            watch loui ck epic answer trump vs hillari
3               trump ask common daughter answer creepi
4            presid elect trump new era unpredict await
5                             trump busi indonesia make
6               constanc hall judgey women donald trump
7                trump leak truth power war win analysi
8     donald trump presid news australian economi de...
9             kevin macdonald celebr trump amaz victori
10    sur twitter donald trump se f licit avoir vot ...
11                     concern rais press freedom trump
12                   investor welcom donald trump worri
13    berni sander could replac presid trump littl k...
14    us women open golf may yet move donald trump o...
15                  donald trump janet yellen head head
16    donald trump muslim ban immedi consequ zoe

Evet, gerçekten veri kümesi içerisinde sadece köklerin olduğunu ve herhangi bir bağlaç kalmadığını görebiliyoruz. Tam olarak istediğimiz durum buydu. Artık veri kümesini bilgisayarın anlayabileceği 0 ve 1 lerden oluşan bir matris haline getirebiliriz.

Bunun için kullandığımız Python kütüphanesi Scikit-Learn. Bu kütüphanenin içerisindeki CountVectorizer sınıfını kullanarak bu işlemi yapabiliyoruz. Bu sınıf veri kümesinde ki bütün kelimeleri alıp birer değişken haline getiriyor ve her bir cümlede o kelimenin olup olmadığına göre 0 ve 1 sayısını veriyor. En nihayetinde elimizde çok fazla sayıda 0 ve çok az sayıda 1’lerden oluşan bir sparse matris oluşuyor.


In [8]:
# Vectorized All Data
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(sentenceList)

result = pd.DataFrame(data=X.toarray())

print(result)

      0     1     2     3     4     5     6     7     8     9     ...   3844  \
0        0     0     0     0     0     0     0     0     0     0  ...      0   
1        0     1     0     0     0     0     0     0     0     0  ...      0   
2        0     0     0     0     0     0     0     0     0     0  ...      0   
3        0     0     0     0     0     0     0     0     0     0  ...      0   
4        0     0     0     0     0     0     0     0     0     0  ...      0   
5        0     0     0     0     0     0     0     0     0     0  ...      0   
6        0     0     0     0     0     0     0     0     0     0  ...      0   
7        0     0     0     0     0     0     0     0     0     0  ...      0   
8        0     0     0     0     0     0     0     0     0     0  ...      0   
9        0     0     0     0     0     0     0     0     0     0  ...      0   
10       0     0     0     0     0     0     0     0     0     0  ...      0   
11       0     0     0     0     0     0

Şu noktadan itibaren bu veri kümesi artık bilgisayarın anlayabileceği bir model haline geldi.

Test veri kümemiz ile eğitim veri kümemiz birbirinden tamamen ayrı. Bu nedenle bu işlemlerin birebir aynısı test veri kümesi içinde uygulanmıştır.


In [0]:
# Test Data
testSentenceList = []
for i in range(len(testList)):
    # Regular Expressions
    comment = re.sub('[^a-zA-Z]', ' ', testList[i])
    # Upper Case Problem
    comment = comment.lower()
    # Splitting
    comment = comment.split()
    # Stemming and Stop Words
    comment = [ps.stem(kelime) for kelime in comment if not kelime in set(stopwords.words('english'))]
    
    comment = ' '.join(comment)
    testSentenceList.append(comment)

X = vectorizer.transform(testSentenceList)

testResult = pd.DataFrame(data=X.toarray())
# PCA for Test
#testResult = pca.transform(testResult)


## Derin Öğrenme
Nihayet artık veri kümesi bir model oluşturmaya hazır hale gelmiştir. Modelimiz bir derin öğrenme modelidir. Dolayısıyla derin öğrenmenin ne olduğundan kısaca bahsetmekte fayda var. Derin öğrenme, basit yapay sinir ağlarının daha karmaşık hale getirilmişi ve her bir katmana istediğimiz gibi müdahale edebildiğimiz bir yapısıdır. Karmaşıklık arttıkça derin öğrenmeye olan ihtiyaçta artıyor.


[![image](https://i.hizliresim.com/9a0AGO.jpg)](https://hizliresim.com/9a0AGO)

Şimdi nasıl bir model tasarladığıma kısaca bir bakalım. Elimizde 1 tane giriş katmanı, 1 tane saklı katman ve 2 tane seyreltme katmanı bulunmaktadır. Giriş ve saklı katman 1500’er tane nörona sahiptir. 2 tane seyreltme katmanımız var ve bu katmanlar 0.5 eşik değerine sahip. Bu seyreltme katmanlarının temel amacı ezberlemeyi azaltmaktır. Bu istemediğimiz bir durum. Dolayısıyla bu seyreltme katmanları eşik değerini 0.5 olarak girdiğimiz için burada 1500 tane olan nöron sayısını rasgele olarak yarıya düşürüyor. Bu noktadan sonra sonucu gerçek ya da yalan olarak belirlenecek olan tek bir nörona sahip bir tane çıkış katmanı oluşturuluyor. Daha geniş bir açıdan baktığımız zaman 1 tane giriş katmanımız, 1 tane saklı katmanımız ve 1 tanede çıkış katmanımız oluşuyor. Şimdi bunun nasıl kodlandığına bakalım.


In [0]:
from keras.layers import Dropout
from keras.models import Sequential
from keras.layers import Dense, LSTM
from keras.callbacks import TensorBoard
from keras import optimizers
import datetime

time = datetime.datetime.now().strftime("%Y_%m_%d_%H_%M_%S")

kerasboard = TensorBoard(log_dir="/tmp/tensorboard/{}".format(time))

#result = result.reshape((2777, 1, 2000))
#testResult = testResult.reshape((489, 1, 2000))

classifier = Sequential()

classifier.add(Dense(1500, kernel_initializer="glorot_normal", activation='sigmoid', input_dim=3854)) #1 -> Giriş Katmanı
#classifier.add(LSTM(1500, input_shape=(1, 2000), activation='sigmoid',return_sequences=False))
classifier.add(Dropout(0.50))


classifier.add(Dense(1500,kernel_initializer='glorot_normal' ,activation='sigmoid')) #2 -> Saklı Katman
classifier.add(Dropout(0.50))


classifier.add(Dense(1, kernel_initializer="glorot_normal", activation='sigmoid')) #3 -> Çıkış Katmanı

adam = optimizers.Adam(lr=0.0001, decay=0.00008)

classifier.compile(optimizer=adam,loss='binary_crossentropy', metrics=['accuracy'])


Öncelikle kullanılan kütüphane Google’ın bir çalışanı tarafından tasarlanan Keras kütüphanesidir. Bu kütüphane arka planında Tensorflow denen başka bir kütüphaneyi kullanmaktadır. Bu kütüphaneyi ise Google şirketinin direk kendisi tasarladı. Bu iki kütüphane sayesinde Python üzerinden Derin öğrenme modelleri oluşturabiliyoruz. Bunlardan başka kütüphanelerde vardır fakat en çok kullanılan iki kütüphane bu ikisidir.

Burada odaklanmamız gereken kısımlar Dense ve Dropout fonksiyonlarının olduğu kısımlardır. 

**Giriş Katmanı -> ** 1 numaralı kısımda ilk parametre olarak 1500 sayısını girerek nöron sayısını girmiş olduk. 

İkinci parametre kernel_initializer başlangıçta hangi istatistiksel dağılımı kullanarak nöronlara ağırlıklar vereceğimizi belirlediğimiz kısım. Ağırlık verirken dikkat edilmesi gereken 3 tane husus vardır.
1. Her bir optimizasyon gerçekleştiğinde ağırlıkların ortalamasının sıfıra eşit olması gerekir.
2. Bütün ağırlıkların varyanslarının eşit olması gerekmektedir yani değişen varyanslılık sorununun olmamasıdır.
3. Ağırlıkların kesinlikle sıfıra eşit olmaması ve ne çok küçük değerler ne de çok büyük değerlere eşit olmasıdır.

Bütün bu hususların ışığında en çok kullanılan 2 tane initializer Xavier ve He'dir. Ben çalışmamda Xavier(glorot_normal) kullandım.

Diğer bir parametre aktivasyon fonksiyonudur, bu fonksiyon girişteki katsayıyı alıp kendi fonksiyonunda işlem uygulayarak çıktı katsayısı elde etmemizi sağlıyor. Daha sonra bu çıktı katsayısı ya direk sonucun kendisi olur ya da bir başka katmanın girdisi olur.
Aktivasyon fonksiyonlarının 2 tane ana özelliği vardır.
1. Derin Öğrenme kullanırken elimizdeki veri setinin karmaşıklığı o veri setinin çok büyük bir ihtimalle doğrusal olmayacağını ön görür. Bu sebeple bu fonksiyonlar doğrusal olmayan fonksiyonlardır.
2. Her bir geri beslemede bu aktivasyon fonksiyonlarının türevi alınarak en optimal sonuca ulaşmak amaçlanır. Bu nedenle kolay türevi alınabilen fonksiyonlar tercih edilir böylece zaten sistemi çok  yoran derin öğrenme algoritmalarının iş yükü biraz daha hafifletilmiş olur.

Burada aktivasyon fonksiyonu olarak en popüleri olan **Sigmoid** fonksiyonu kullanmayı tercih ettim.

En son parametre olarakta elimizdeki veri kümesinin değişken sayısını vermemiz gerekiyor. Bu bilgi veri kümesindeki kelime sayısına eşittir yani 3854.

Daha sonra Dropout katmanını oluşturuyoruz ve eşik değeri olarak 0.5 giriyoruz. 

**Saklı Katman -> ** 2 numaralı kısımda 1 numaralı kısımda yaptığımız şeyden farklı olan hiçbir şey yapmıyoruz. 

**Çıkış Katmanı -> ** 3 numaralı kısımda ise sonucu doğru ya da yalan olan 1 tane nörona sahip bir katman oluşturuyoruz.

Bundan sonraki aşamada bir optimizasyon fonksiyonu belirlememiz gerekiyor. Bu fonksiyonun amacı ise her bir geri beslemede nöronların katsayılarını daha optimal hale getirip doğruluk yüzdemizi arttırmayı amaçlamak. Bakalım kodlarımız nasıl çalışmış.

In [21]:
classifier.fit(result, targetInputFloat, epochs=40, callbacks=[kerasboard])

Epoch 1/40
2777/2777 [==============================] - 16s 6ms/step - loss: 0.7630 - acc: 0.5290
Epoch 2/40
2777/2777 [==============================] - 15s 6ms/step - loss: 0.7226 - acc: 0.5574
Epoch 3/40
2777/2777 [==============================] - 15s 5ms/step - loss: 0.7232 - acc: 0.5582
Epoch 4/40
2777/2777 [==============================] - 15s 5ms/step - loss: 0.6978 - acc: 0.5834
Epoch 5/40
2777/2777 [==============================] - 15s 5ms/step - loss: 0.6855 - acc: 0.5870
Epoch 6/40
2777/2777 [==============================] - 15s 6ms/step - loss: 0.6614 - acc: 0.6169
Epoch 7/40
2777/2777 [==============================] - 15s 6ms/step - loss: 0.6643 - acc: 0.6060
Epoch 8/40
2777/2777 [==============================] - 16s 6ms/step - loss: 0.6465 - acc: 0.6097
Epoch 9/40
2777/2777 [==============================] - 15s 5ms/step - loss: 0.6298 - acc: 0.6421
Epoch 10/40
2777/2777 [==============================] - 15s 5ms/step - loss: 0.6143 - acc: 0.6485
Epoch 11/40
2777/27

Modellemede toplamda 40 tane geri besleme yani optimizasyon çalışması yapıldı. Loss yani hata değerlerinin giderek düşmesini bekliyoruz gerçekten de öyle olmuş. Accuracy yani doğruluk yüzdelerinin de her bir geri beslemede artmasını amaçlıyoruz. Oda artmış, ilk başta %52 doğru tahmin yapabilirken 40. geri beslemede %89 oranına çıkmış. Bu oranların hepsi eğitim veri kümesi için, bakalım modele test veri kümesini verdiğimizde nasıl bir sonuç alacağız en nihayetinde bizim için asıl önemli olan kısım modelin daha önce bilmediği bir cümleyi nasıl analiz ettiği.


In [22]:
prediction = classifier.predict(testResult)

prediction = (prediction > 0.5)

from sklearn.metrics import accuracy_score

acScoreNN = accuracy_score(lb.fit_transform(testDataForComparison), prediction)
print("Accuracy : " + str(acScoreNN))

probaNN = classifier.predict_proba(testResult)
print("NN AUC " + str(metrics.roc_auc_score(lb.fit_transform(testDataForComparison), probaNN)))

print("tensorboard --logdir="+kerasboard.log_dir)

/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/label.py:235: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Accuracy : 0.8220858895705522
NN AUC 0.9125109208457103
tensorboard --logdir=/tmp/tensorboard/2019_04_21_18_28_31


Sonuçlarda görüldüğü gibi model daha önce bilmediği bir veri kümesiyle karşılaştığı zaman %83 doğru tahmin yüzdesine ulaşmış.

ROC Eğrisi altında kalan alan %91 çıkmış. Bundan çıkan yorum modelin %91 oranında doğruyla yanlışı ayırt edebilme gücüne sahip olduğudur. Bu değer 1’e ne kadar yakın olursa model o kadar iyi demektir. Tabi ROC Eğrisi sonucunun doğruluk yüzdesine göre daha yanıltıcı sonuç verebileceğini söylemekte fayda var.


Şimdi oluşturmuş olduğumuz derin öğrenme modelinin her bir geri beslemede doğruluk yüzdesini nasıl beslediğini TensorBoard ortamında görselleştirelim.

In [0]:
# Load the TensorBoard notebook extension
%load_ext tensorboard.notebook

#Activate Tensorboard
%tensorboard --logdir=/tmp/tensorboard/2019_04_21_18_28_31

TensorBoard'u çağırmak için derin öğrenme modelini kurarken oluşturduğumuz log adresini kullanıyoruz. Normal şartlarda bu log adresinin farklı bir tarayıcı sekmesinde açılmasını bekleriz fakat Jupyter Notebook'a yeni gelmiş bir eklenti sayesinde bunu artık kod hücresinin içerisinde açabiliyoruz. (tensorboard.notebook eklentisi)

Bundan sonraki aşamada oluşturmuş olduğumuz derin öğrenme modelini Pickle kütüphanesini kullanarak kaydediyoruz. Böylece ne zaman istersek model üzerinden tahminler gerçekleştirebileceğiz.

In [0]:
#Saving the best model with Pickle (Neural %84.04)
import pickle
pickle.dump(classifier, open("NeuralNews", 'wb'))

loading = pickle.load(open("NeuralNews", 'rb'))
predictionPickleNeural = loading.predict(testResult)

predictionPickleNeural = (predictionPickleNeural > 0.5)

acScorePickleNeural = accuracy_score(lb.fit_transform(testDataForComparison), predictionPickleNeural)
print("Accuracy Pickle Neural : " + str(acScorePickleNeural))